# Unit Commitment

This tutorial runs through examples of unit commitment for generators at a single bus. Examples of minimum part-load, minimum up time, minimum down time, start up costs, shut down costs and ramp rate restrictions are shown, as well as how to set up a rolling horizon optimization.

To enable unit commitment on a component (`Link` or `Generator`), set its attribute `committable=True`.

Remember:
1. Minimum part load $\rightarrow$ ON means must generate
2. Minimum up time $\rightarrow$ ON means must stay ON
3. Minimum down time $\rightarrow$ OFF means must stay OFF
4. Binary status $\rightarrow$ couples decisions over time

In [1]:
import pandas as pd

import pypsa

## Minimum Part Load

In the final snapshot, the load goes below the part-load limit of the coal generator (30%), forcing gas to commit.

**What PyPSA is enforcing:**

Because the generators are committable, PyPSA introduces:
1. Binary on/off status
2. Minimum generation when on
3. Startup/shutdown logic

So when a generator is on, it must respect: `p >= p_min_pu` $\times$ `p_nom`

**Snapshots 0-2: Coals stays on**

Since load is always above 3,000 MW, coal can stay committed, adjust output to exactly match load, and avoid expensive gas entirely. 

**Snapshot 3:**

If coal stayed on with its minimum output of 3,000 MW, that would over-generate by 2,200 MW. It has no place to dump power so this source is infeasible. Keeping cola on violates the minimum part load constraint. Hence, you shut down coal and turn on gas as it can produce exactly the required load. Although it has hgiher cost, it is a more feasible option. 

### Mathematical formulation (standard MILP):

**Decision variables:**

1. Generator commitment (on/off): $u_{g,t} \in {0,1}$
    * This is created automatically when ```committable=True```

2. Generator output: $p_{g,t} \geq 0$
    * Electrical output in MW, continuous variable

3. Startup/shutdown indicators:
$$y_{g,t} \in {0,1} startup at t$$
$$z_{g,t} \in {0,1} shutdown at t$$
* These track transitions, needed for minimum up/down time logic

**Minimum part load (```p_min_pu```)**
* Let $P^{max}_g$ = ```p_nom```
* Let $\underbar{p}_g$ = ```p_min_pu```
Constraint: $$\underbar{p}_g P^{max}_g u_{g,t} \leq p_{g,t} \leq P^{max}_g u_{g,t}$$
Interpretation: 
* If the generator is OFF ($u=0$): $p_{g,t} =0$
* If the generator is ON ($u=1$): $\underbar{p}_g P^{max}_ \leq p_{g,t} \leq P^{max}_g$
* This is any coal is infeasible at 800 MW load adn gas has to take over instead.

**Objective:**
$$\min \sum_t \left(c_g p_{g,t} + c_g^{standby} u_{g,t} + c_g^{start} y_{g,t}\right)$$
* $c_g =$ ```marginal_cost```
* $c_g^{standby}=$ ```stand_by_cost```
* $c_g^{start}=$ optional startup cost

In [2]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    p_nom=10_000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    p_nom=1_000,
)

nu.add("Load", "load", bus="bus", p_set=[4_000, 6_000, 5_000, 800])

In [3]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 3.13s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 36 duals
Objective: 3.56e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [4]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


In [5]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


## Minimum Up Time

Gas has a minimum up time, forcing it to be online longer than otherwise necessary, which incurs a standby cost for status up without generation.

If gas starts at snapshot 1, it would have to remain on in snapshots 1, 2, 3. That would force gas to be on in snapshot 3, where coal alone can meet the load more cheaply. So PyPSA starts gas one snapshot earlier, at snapshot 0, to align the 3-period up-time window optimally.

Gas generates only 100 MW in snapshots 0 & 2 becase when it is ON, it must respect $p ≥ p_min_pu × p_nom = 100$ MW. So PyPSA keeps gas at minimum output in these snapshots, which minimizes expensive gas usage while respecting commitment constraints.

```stand_by_cost=50``` is incurred whenever gas is ON, regardless of output. It is for each snapshot separately. 

### Additional components to the mathematical formulation:

**Commitment transition equations:**

Startups and shutdowns
$$u_{g,t} -u_{g,t-1} = y_{g,y} - z_{g,t} $$

This constraint ensures:
* you can't start and stop at the same time
* Commitment status evolves correctly

**Minimum up time constraint (```min_up_time```):**

Let $UT_g=$ minimum up time (in snapshots)
$$\sum^t_{\tau=t-UT_g+1} y_{g,\tau} \leq u_{g,t} \quad \forall t$$
Interpretation:
* If a startup occured in the last $UT_g$ periods, $u_{g,t}$ must equal 1
* This is why gas stayed online even when it produced only 100 MW

**Initial conditions:**

If generator was ON before the horizon, $u_{g,t}=1$. And the remaining up time is:
$$UT_g^{remaining} = \max(0,UT_g-up\_time\_before)$$

In [2]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    stand_by_cost=50,
    marginal_cost=70,
    p_min_pu=0.1,
    up_time_before=0,
    min_up_time=3,
    p_nom=1_000,
)

nu.add("Load", "load", bus="bus", p_set=[4_000, 800, 5_000, 3_000])

In [3]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 3.13s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 39 duals
Objective: 3.06e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-up-time, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [8]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,1.0,1.0
1,0.0,1.0
2,1.0,1.0
3,1.0,0.0


In [9]:
nu.objective

306150.0

In [10]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,3900.0,100.0
1,0.0,800.0
2,4900.0,100.0
3,3000.0,-0.0


## Minimum Down Time

Coal has a minimum down time, forcing it to go off longer than otherwise cost-optimal.

**Constraints added:**
```
min_down_time = 2
down_time_before = 1
```

Meaning for coal:
* If coal shuts down, it must stay off for at least 2 snapshots
* At snapshot 0, coal is treated as having already been down for 1 snapshot
* So if it is off at snapshot 0, it must also be off at snapshot 1

Even though coal could meet the load in snapshot 0, it cannot be turned on, because doing so would violate the down-time constraint around snapshot 1.

NOTE: Ideally should specify ```up_time_before``` if specifying ```down_time_before=1```. 

### Additional components to the mathematical formulation:

**Minimum down time constraint (```min_down_time```)**

Let $DT_g =$ minimum down time
$$\sum^t_{\tau=t-DT_g+1} z_{g,\tau} \leq 1-u_{g,t} \quad \forall t$$
Interpretation:
* If a shutdown happened within the last $DT_g$ periods, $u_{g,t}$ must equal 0.
* This is what forced coal to stay off for two full snapshots, even though it was cheap.

**Initial conditions:**

If generator was OFF before the horizon, $u_{g,0}=0$. And the remaining down time is:
$$DT_g^{remaining} = \max(0,DT_g-down\_time\_before)$$

In [4]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    down_time_before=1,
    p_nom=10_000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    p_nom=4_000,
)

nu.add("Load", "load", bus="bus", p_set=[3_000, 800, 3_000, 8_000])

In [5]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.06s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 40 duals
Objective: 4.86e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-down-time, Generator-com-status-min_up_time_must_stay_up, Generator-com-status-min_down_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [6]:
nu.objective

486000.0

In [7]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [8]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,0.0,3000.0
1,0.0,800.0
2,3000.0,0.0
3,8000.0,0.0


## Start Up and Shut Down Costs